In [1]:
import collections

In [2]:
def parse_input(filename):
    with open(filename) as f:
        lines = [line.strip() for line in f]
    
    module_types = dict()
    module_targets = dict()
    module_state = dict()
    
    for line in lines:
        src, targets = line.split(" -> ")
        if src == "broadcaster":
            tpe = 'b'
        else:
            tpe = src[0]
            src = src[1:]
        module_types[src] = tpe
        module_targets[src] = [t.strip() for t in targets.split(",")]
    module_types['button'] = 'b'
    module_targets['button'] = ['broadcaster']
    
    return module_types, module_targets

In [3]:
# module_types, targets = parse_input("../input/day20-sample1.txt")
# module_types, targets = parse_input("../input/day20-sample2.txt")
module_types, targets = parse_input("../input/day20-input.txt")

In [4]:
# module_types

In [5]:
# targets

In [6]:
def find_inputs(mod):
    return [src for src, ts in targets.items() if mod in ts]

state = dict()
for mod, tpe in module_types.items():
    if tpe == '%': # flipflop
        state[mod] = False
    elif tpe == '&': # conjunction
        state[mod] = {m: False for m in find_inputs(mod)}

In [7]:
# state

In [8]:
Q = collections.deque()

In [9]:
xmstate = []

In [10]:
def loop(debug = False, loopnr = 0):
    # button push
    Q.append(('button', 'broadcaster', False)) # src, target, hi
    counter_lo = 0
    counter_hi = 0
    while Q:
        (src, mod, hilo) = Q.popleft()
        if hilo:
            counter_hi += 1
        else:
            counter_lo += 1
        
        if debug:
            print(f"{src} -{'high' if hilo else 'low'}-> {mod}")
        
        if mod == 'rx':
            # print("heywow it's rx", loopnr, src, mod, hilo)
            if any(state['xm'].values()):
                xmstate.append((loopnr, state['xm'].copy()))

        if mod not in module_types:
            # simple module with no outputs. end here
            continue

        tpe = module_types[mod]
        pulse = hilo

        if tpe == '%': # flipflop
            if hilo:
                # ignore, send nothing
                continue
            else:
                s = state[mod]
                pulse = not s
                state[mod] = not s
        elif tpe == '&': # conj
            mem = state[mod]
            mem[src] = hilo
            pulse = not all(m for m in mem.values())

        # send new pulses
        for t in targets[mod]:
            Q.append((mod, t, pulse))
    return counter_lo, counter_hi

In [11]:
def part1():
    count_lo, count_hi = 0, 0
    for i in range(1000):
        lo, hi = loop()
        count_lo += lo
        count_hi += hi
    return count_lo * count_hi
part1()

788081152

In [12]:
'rx' in [t for src, ts in targets.items() for t in ts]

True

In [13]:
xmstate = []

for i in range(50000):
    loop(False, i)

In [14]:
module_types['xm']

'&'

In [15]:
find_inputs('xm')

['ft', 'jz', 'sv', 'ng']

In [16]:
xm_input_loops = {t: sorted(set([loopnr for loopnr, s in xmstate if s[t]])) for t in find_inputs('xm')}

In [17]:
cycles = []
for mod, idxs in xm_input_loops.items():
    cs = [b - a for a, b in zip(idxs[:-1], idxs[1:])]
    if not all(c == cs[0] for c in cs):
        print("uneven cycles for ", mod)
    cycles.append(cs[0])

In [18]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def lcm(a, b):
    return a * b // gcd(a, b)

from functools import reduce

In [19]:
part2 = reduce(lcm, cycles)
part2

224602011344203